In [9]:
import ipywidgets as widgets
import pandas as pd
from IPython.display import display
import io

file_upload = widgets.FileUpload()

output = widgets.Output()
textInput = widgets.Text()

def on_upload_change(change):
    # We need output Widget to get display() results in the code, else nothing will appear. In modern ipywidgets and Jupyter you have to be explicit in how you connect up the output being generated 
    # to the output area of the cell'.
    with output:
        output.clear_output()
        global file_contents      # We had to make it a global variable, as we need it to create 'df' in 'read_excel_sheet()'.
        file_contents = change.new[0].content
        #df = pd.read_excel(io.BytesIO(change.new[0].content),sheet_name=sheet_dropdown.value,dtype=str)
        #display(change)

# Function to read Excel sheet and create a DataFrame
def read_excel_sheet(file_content, sheet_name):
     df = pd.read_excel(io.BytesIO(file_content), sheet_name=sheet_name)
     return df

# Create Dropdown widget for sheet selection
sheet_dropdown = widgets.Dropdown(options=[], description='Select Sheet')

# Function to update dropdown options when file is uploaded
def update_sheet_dropdown(change):
    excel_file = pd.ExcelFile(io.BytesIO(change.new[0].content))
    sheet_dropdown.options = excel_file.sheet_names
    sheet_dropdown.value = None # This needs to be done else the next time we choose a file, 
                                # you won't get an empty dropdown and a sheet would be selected
                                # by default, which is problematic because we can only create 
                                # a 'df' once an event for choosing a sheet is triggered.

# Function to handle sheet selection and DataFrame creation
def handle_sheet_selection(change):
    with output:
        #output.clear_output()
        #display(change)
        #display(change.new)
        sheet_name = change.new
        dataframe = read_excel_sheet(file_contents, sheet_name)
        dataframe["Order Amount"] = dataframe["Order Amount"].apply(lambda x: x.replace('VND', '')).apply(lambda x: x.replace('.', ''))

        productNameToFilter = "BỘT TRONG VEO"
        createDateToFilter = "31/08/2024"
        fileName = "{}_{}.xlsx".format(productNameToFilter.replace(" ", ""), createDateToFilter.replace("/", ""))

        checkContainProductName = dataframe["Product Name"].str.contains(productNameToFilter)
        checkContainCreatedDate = dataframe["Created Time"].str.contains(createDateToFilter)

        df_demo = dataframe.loc[checkContainProductName & checkContainCreatedDate]
        df_demo['Order Amount'] = pd.to_numeric(df_demo['Order Amount'])
        totalAmount = df_demo['Order Amount'].sum()
        display(totalAmount)
        df_demo.to_excel(fileName)
        # display(df_demo)

file_upload.observe(update_sheet_dropdown, names='value')
sheet_dropdown.observe(handle_sheet_selection, names='value')
file_upload.observe(on_upload_change, names='value')

boxes = widgets.HBox([file_upload,sheet_dropdown])
display(boxes)

display(output)


Output()